# 5. Neural network trained on base and additional features.


In [ ]:
from numpy.random import seed
seed(1)
from tensorflow.keras.utils import set_random_seed
set_random_seed(2)

In [ ]:
import pandas as pd
import numpy as np
import pickle

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
plt.style.use('dark_background')

In [ ]:
from keras import Sequential, layers, models

In [ ]:
X_train, Y_train, X_test, Y_test=pickle.load(open("staley16+addtl_feats_split.pkl","rb"))
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

In [ ]:
use_feats = X_train.columns

In [ ]:
X_train

In [ ]:
import pickle

In [ ]:
ssc=pickle.load(open("feature_scaler.pkl", "rb"))
X_train_s=ssc.transform(X_train)
X_test_s=ssc.transform(X_test)
print(X_train_s.shape, X_test_s.shape)

In [ ]:
def getModel(nins, drop=0.3):
    inputs = layers.Input(shape=(nins,))
    
    x = layers.Dense(32, activation=None, kernel_initializer='glorot_normal')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)    
    x = layers.Dropout(drop)(x) #.1
    
    x = layers.Dense(48, activation=None, kernel_initializer='glorot_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('tanh')(x)    
    x = layers.Dropout(drop)(x)       #.1   
    
    x = layers.Dense(92, activation=None, kernel_initializer='glorot_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)    
    x = layers.Dropout(drop)(x)       #.1
        
    x = layers.Dense(196, activation=None, kernel_initializer='glorot_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('tanh')(x)    
    x = layers.Dropout(drop)(x)       #.1
    
    x = layers.Dense(392, activation=None, kernel_initializer='glorot_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)    
    x = layers.Dropout(drop)(x)       #.1
    
    x = layers.Dense(196, activation=None, kernel_initializer='glorot_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('tanh')(x)    
    x = layers.Dropout(drop)(x)       #.1
    
    x = layers.Dense(92, activation=None, kernel_initializer='glorot_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)    
    x = layers.Dropout(drop)(x)   
    
    x = layers.Dense(48, activation=None, kernel_initializer='glorot_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('tanh')(x)    
    x = layers.Dropout(drop)(x)       #.1
    
    x = layers.Dense(32, activation=None, kernel_initializer='glorot_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)     
    x = layers.Dropout(drop/2)(x) #.05
    
    x = layers.Dense(16, activation=None, kernel_initializer='glorot_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('tanh')(x)    
    #x = layers.Dropout(0.05)(x)
    
    x = layers.Dense(8, activation=None, kernel_initializer='glorot_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)    
    #x = layers.Dropout(drop/8)(x)
    predictions = layers.Dense(1, activation='sigmoid')(x)
    
    model = models.Model(inputs=inputs, outputs=predictions)
    return model

In [ ]:
model=getModel(len(use_feats), drop=0.275)

In [ ]:
import tensorflow as tf

In [ ]:
%%time
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='binary_crossentropy',metrics=['accuracy','mae','BinaryIoU','AUC'])  #msle
history = model.fit(X_train_s, Y_train, validation_split=0.0, epochs=2000, 
                    batch_size=100, verbose=0)

In [ ]:
for key in history.history.keys():
    print(key, history.history[key][-1])

In [ ]:
fig,ax=plt.subplots(ncols=3, figsize=(12,3))

for n,metric in enumerate(["loss", "mae", "binary_io_u"]):
    ax[n].plot(history.history[metric], label='train')
    #ax[n].plot(history.history["val_" + metric], label='test')
    ax[n].set_title(metric)
ax[0].legend()

In [ ]:
Y_train_pred=model.predict(X_train_s)

In [ ]:
jaccard_score(Y_train, np.round(Y_train_pred)), roc_auc_score(Y_train, Y_train_pred)
#print(accuracy_score(Y_train, np.round(Y_train_pred)))

In [ ]:
Y_test_pred=model.predict(X_test_s)

In [ ]:
jaccard_score(Y_test, np.round(Y_test_pred)), roc_auc_score(Y_test, Y_test_pred)

In [ ]:
from sklearn.metrics import roc_curve, RocCurveDisplay

In [ ]:
fig,ax=plt.subplots()
RocCurveDisplay.from_predictions(Y_train, Y_train_pred, ax=ax, linewidth=1, 
                                 color='red', name='train', linestyle="dashed")
RocCurveDisplay.from_predictions(Y_test, Y_test_pred, ax=ax, linewidth=1, color='yellow', name='test')

In [ ]:
Y_test_proba=np.zeros((len(Y_test_pred),2), dtype=float)
Y_test_proba[:,0] = 1.-Y_test_pred[:,0]
Y_test_proba[:,1] = Y_test_pred[:,0]

In [ ]:
from os.path import isdir
from posix import mkdir

In [ ]:
odir="totaccum_sediment_susc_frac_randn"

if not isdir(odir):
    mkdir(odir)
    
ofile=odir + "/NN_proba.pkl"
pickle.dump(Y_test_proba, open(ofile,"wb"))

Re-train using all the data:

In [ ]:
X_train_s.shape, Y_train.shape

Merge training and testing set, and use to re-train model.

In [ ]:
X_traintest_s=np.append(X_train_s, X_test_s, axis=0)
Y_traintest=np.append(Y_train, Y_test)
print(X_traintest_s.shape, Y_traintest.shape)

In [ ]:
modelp=getModel(len(use_feats), drop=0.275)

In [ ]:
optp = tf.keras.optimizers.Adam(learning_rate=0.001)
modelp.compile(optimizer=optp, loss='binary_crossentropy',metrics=['accuracy','mae','BinaryIoU','AUC'])  #msle
historyp = modelp.fit(X_traintest_s, Y_traintest, validation_split=0.0, epochs=1000, 
                    batch_size=100, verbose=0)

In [ ]:
for key in historyp.history.keys():
    print(key, historyp.history[key][-1])

In [ ]:
Y_traintest_pred=modelp.predict(X_traintest_s)
jaccard_score(Y_traintest, np.round(Y_traintest_pred)), roc_auc_score(Y_traintest, Y_traintest_pred)

Save model for debris flow forecasting:

In [ ]:
modelp.save('NNp_11layer_minpars_accum_suscfrac_randn.h5')

In [ ]:
!md5sum *.h5